# Veri Seti Hikayesi
Vehicle Sales and Market Trends" veri seti, araç satış işlemleri ve pazar eğilimlerine dair kapsamlı bir bilgi sunmaktadır. Veri seti, çeşitli araçların satış işlemleriyle ilgili detayları içermektedir. Araçların markası, modeli, donanım seviyesi, gövde tipi, üretim yılı gibi temel bilgilerden, satış fiyatı, satış tarihi, kayıtlı olduğu eyalet ve satıcı bilgilerine kadar geniş bir yelpazede veri sağlamaktadır. Ayrıca araçların durumu, kilometre bilgisi, dış ve iç renk seçenekleri gibi faktörler de veri setinde yer almaktadır.



# Değişkenler

- **Yıl (Year):** Aracın üretildiği yıl.
- **Marka (Make):** Aracın markası (ör. Toyota, Ford).
- **Model (Model):** Aracın modeli (ör. Corolla, F-150).
- **Donanım (Trim):** Aracın donanım seviyesi.
- **Gövde Tipi (Body Type):** Aracın gövde tipi (ör. sedan, SUV).
- **Şanzıman Tipi (Transmission Type):** Aracın şanzıman tipi (otomatik, manuel).
- **VIN (Vehicle Identification Number):** Aracın araç kimlik numarası (VIN).
- **Kayıtlı Eyalet (State of Registration):** Aracın kayıtlı olduğu eyalet.
- **Durum Puanı (Condition Rating):** Aracın durumu hakkında puanlama.
- **Kilometre (Odometer Reading):** Aracın kilometre bilgisi.
- **Dış Renk (Exterior Color):** Aracın dış rengi.
- **İç Renk (Interior Color):** Aracın iç rengi.
- **Satıcı Bilgileri (Seller Information):** Aracın satışını yapan satıcıya ait bilgiler.
- **Manheim Pazar Raporu (MMR) Değeri (Manheim Market Report Value):** Aracın piyasa değeri (Manheim tarafından belirlenen değer).
- **Satış Fiyatı (Selling Price):** Aracın satıldığı fiyat.
- **Satış Tarihi (Sale Date):** Aracın satıldığı tarih.
  



# Kütüphanelerin Yüklenmesi

In [ ]:
############ LIBRARIES ############

# BASE
import numpy as np  # Sayısal işlemler ve diziler üzerinde matematiksel hesaplamalar için kullanılır.
import pandas as pd  # Veri işleme ve veri çerçeveleri (DataFrame) üzerinde çalışmak için kullanılan güçlü bir kütüphane.
import seaborn as sns  # Veri görselleştirme kütüphanesi, özellikle istatistiksel grafikler oluşturmak için kullanılır.
from matplotlib import pyplot as plt  # Grafik çizimi için kullanılan temel kütüphanelerden biridir.

# DATA PREPROCESSING
from sklearn import preprocessing  # Veri ölçekleme, etiket kodlama gibi ön işleme işlemleri için kullanılır.
from sklearn.neighbors import LocalOutlierFactor  # Aykırı değer tespiti için kullanılan bir yöntem.


# WARNINGS
import warnings  # Uyarıları yönetmek için kullanılan Python kütüphanesi.
warnings.filterwarnings("ignore", category=DeprecationWarning)  # DeprecationWarning'leri görmezden gelir.
warnings.filterwarnings("ignore", category=FutureWarning)  # FutureWarning'leri görmezden gelir.
pd.set_option('display.max_columns', None)  # Pandas'ta tüm sütunların görünmesi için ayar.
pd.set_option('display.width', None)  # Satır genişliğini sınırsız yapar.
pd.set_option('display.float_format', lambda x: '%.3f' % x)  # Sayıların ondalıklı olarak gösterilmesi için format belirler.


# Veriye Genel Bakış

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
file_path = "/kaggle/input/vehicle-sales-data/car_prices.csv"
df = pd.read_csv(file_path)
df.head()

In [ ]:
# Genel bakış için bir fonksiyon yazalım. 
def checkDf(dataframe, head = 8):
  print("######################## Shape ########################")
  print(dataframe.shape)
  print("\n######################## Types ########################")
  print(dataframe.dtypes)
  print("\n######################## Tail ########################")
  print(dataframe.tail(head))
  print("\n######################## Head ########################")
  print(dataframe.head(head))
  print("\n######################## Null Analysis ########################")
  print(dataframe.isnull().sum())
  print("\n######################## Quantiles ########################")
  print(dataframe.describe([0,0.05, 0.50, 0.95, 0.99, 1]).T)  
  print("\n######################## Unique Values ########################")
  print(dataframe.nunique())

In [ ]:
# Oluşturduğumuz fonksiyonu çalıştıralım ve veriye genel bir bakış atalım.
checkDf(df)

### Tekrar eden değişken sayısını kontrol edelim.

In [ ]:
df.duplicated().sum()

In [ ]:
df.isna().sum()


### Tekrar eden veri yok.

In [ ]:
missing_percentage = (df.isnull().sum().sum() / (df.shape[0] * df.shape[1])) * 100
formatted_percentage = round(missing_percentage, 2)
f"Verideki eksik veri sayısı yüzdelik olarak % {formatted_percentage}'dir."

### Tüm veriler içerisinde %1.38'lik verinin eksik olması iyi bir veriye sahip olduğumuzu gösterir. 

# İlk gözlemlerim: 
Odometer, MMR ve sellingprice sütunlarında özellikle çok yüksek ve çok düşük aykırı değerler mevcut. Örneğin, 999,999 km, 182,000 dolar pazar değeri ve 230,000 dolar satış fiyatı gibi değerler veri kümesindeki genel dağılıma kıyasla aykırı gözükmektedir. 

Bunu %99 ve max değeri arasındaki aşırı farktan anlayabiliriz.

Göz ardı edilemeyecek kadar da eksik verilerimiz var ama fazla da eksik değil.



# Keşifsel Veri Analizi (EDA - Exploratory Data Analysis)

### Numerik ve Kategorik Değişkenlerin Yakalanması

In [ ]:
def grabColNames(dataframe, catTh=10, carTh=1000):
    """

    Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.
    Not: Kategorik değişkenlerin içerisine numerik görünümlü kategorik değişkenler de dahildir.

    Parameters
    ------
        dataframe: dataframe
                Değişken isimleri alınmak istenilen dataframe
        catTh: int, optional
                numerik fakat kategorik olan değişkenler için sınıf eşik değeri
        carTh: int, optinal
                kategorik fakat kardinal değişkenler için sınıf eşik değeri

    Returns
    ------
        catCols: list
                Kategorik değişken listesi
        numCols: list
                Numerik değişken listesi
        catButCar: list
                Kategorik görünümlü kardinal değişken listesi

    Examples
    ------
        import seaborn as sns
        df = sns.load_dataset("iris")
        print(grabColNames(df))


    Notes
    ------
        catCols + numCols + catButCar = toplam değişken sayısı
        numButCat catCols'un içerisinde.
        Return olan 3 liste toplamı toplam değişken sayısına eşittir: catCols + numCols + catButCar = değişken sayısı

    """

    # catCols, catButCar
    catCols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    numButCat = [col for col in dataframe.columns if dataframe[col].nunique() < catTh and
                 dataframe[col].dtypes != "O"]
    catButCar = [col for col in dataframe.columns if dataframe[col].nunique() > carTh and
                 dataframe[col].dtypes == "O"]
    catCols = catCols + numButCat
    catCols = [col for col in catCols if col not in catButCar]

    # numCols
    numCols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    numCols = [col for col in numCols if col not in numButCat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'catCols: {len(catCols)}')
    print(f'numCols: {len(numCols)}')
    print(f'catButCar: {len(catButCar)}')
    print(f'numButCat: {len(numButCat)}')
    return catCols, numCols, catButCar


catCols, numCols, catButCar = grabColNames(df)

In [ ]:
f"Kategorik değişklerimiz: {catCols}"

In [ ]:
f"Numerik değişkenlerimiz: {numCols}"

In [ ]:
f"Kategorik görünümlü kardinal değişkenlerimiz: {catButCar}"

### Değişkenlerimizin türünü geçici olarak belirledik. Ancak buradaki değişkenlerimizin tam türünü anlayabilmek için sektör bilgisi gerekli olduğunu düşünüyorum.

## Numerik ve Kategorik Değişkenlerin Analizi

In [ ]:
# Veri setimizdeki kategorik değişkenlerin sınıf sayısını ve oranlarını yazdırıyoruz

def catSummary(dataframe, colName, plot=False):
    # Kategorik verilerin değerlerini ve oranlarını yazdır
    counts = dataframe[colName].value_counts()
    ratio = 100 * counts / len(dataframe)
    
    # Eksik veri sayısı ve oranı
    missing_count = dataframe[colName].isnull().sum()
    missing_ratio = 100 * missing_count / len(dataframe)
    
    summary_df = pd.DataFrame({colName: counts, "Ratio": ratio})
    print(summary_df)
    print(f"Missing Count: {missing_count}")
    print(f"Missing Ratio: {missing_ratio:.2f}%")
    print("##########################################")

    if plot:
        plt.figure(figsize=(20, 8))
        # Kategorileri azalan sıralamada düzenle
        ax = sns.countplot(x=dataframe[colName], data=dataframe, order=counts.index, palette="viridis")
        plt.xticks(rotation=90)  # Etiketleri 90 derece döndür
        plt.show(block=True)

# Dataframe ve kategorik sütunlar tanımlı olduğunu varsayıyoruz
for col in catCols:
    if df[col].dtypes == "bool":
        print(col)
    elif df[col].nunique() < 100:
        catSummary(df, col, True)
    else:
        catSummary(df, col, False)


## Grafikleri yorumlayalım:
### Make 
- Ford en yüksek sayıda araca sahip ve açık ara önde. Yaklaşık 90.000'den fazla bir sayıya sahip.
- Chevrolet ikinci sırada, Ford'a göre daha düşük olmasına rağmen yine de oldukça yüksek, yaklaşık 60.000 civarında.
- Toyota, Nissan ve Dodge da oldukça yüksek sayılarda araca sahip diğer markalar arasında yer alıyor.
- Lexus, Mazda, Cadillac gibi markalar orta seviyede araç sayısına sahipler.
- Daha az bilinen veya daha az tercih edilen markalar ise grafiğin sağ tarafında yer almakta ve çok daha düşük sayılarda. Örneğin, Ferrari, Rolls-Royce, Lotus gibi markalar çok az sayıda görünüyor.

Genel olarak, grafik bize yaygın olarak tercih edilen markalar ile daha nadir olan markalar arasındaki farkı net bir şekilde göstermektedir. Ford ve Chevrolet gibi markalar piyasada oldukça hakimken, Ferrari gibi lüks markaların daha az sayıda olduğu görülüyor.

### Body 

- Sedan açık ara en yüksek sayıda olan gövde tipi. Yaklaşık 200.000'in üzerinde araç sedan gövde tipine sahip.
- SUV gövde tipi ikinci sırada, yaklaşık 125.000 civarında araç bu gövde tipinde.
- Sedan ve SUV tiplerinin ardından Hatchback, Minivan, Wagon gibi daha küçük araç kategorileri geliyor, ancak bunların sayısı çok daha düşük.
- Diğer gövde tipleri (örneğin, Convertible, Crew Cab, SuperCrew) sayıca çok daha az. Grafiğin sağ tarafındaki gövde tipleri ise neredeyse sıfıra yakın sayılarda.

Genel olarak, piyasada en yaygın kullanılan araçların sedan ve SUV tipi olduğu görülüyor. Diğer gövde tipleri, özellikle daha spesifik olanlar (örneğin Quad Cab, Extended Cab), çok daha az sayıda tercih ediliyor.

### Transmission 

- Otomatik vites açık ara en yüksek sayıda tercih edilen vites türü. Yaklaşık 500.000 araç otomatik vitese sahip. 
- Manuel vites türüne sahip araçları 10.000-15.000 civarında. 
- Sedan olarak girilen değerler yanlış sanırım. Sayısı çok çok az bunlar aykırı değer olarak ele alabiliriz. 

Genel olarak, piyasada en yaygın kullanılan araçların otomatik şanzımana sahip olduğu görülüyor.

### State

- En çok araç satılan eyalet Florida. Satılan araç sayısı 80.000'in üzerinde. 
- İkinci sırada Kaliforniya takip ediyor. Satılan araç sayısı 70.000'in üzerinde.
- Diğer en çok araç satılan eyaletler Pensilvanya, Teksas ve Georgia vb. gibi eyaletler takip ediyor.

Genel olarak, bu eyaletlerin nüfus yoğunluğu, ekonomik durumu ve yerel otomobil tercihleri, araç satışlarında önemli bir rol oynuyor.

### Color 

- En çok satan dış araç rengi Siyah. Satılan siyah araç sayısı 100.000'in üzerinde. 
- İkinci sırada en çok satan dış araç rengini Beyaz takip ediyor. Satılan araç sayısı aynı şekilde 100.000'in üzerinde.
- Diğer en çok satan dış araç renkleri Gümüş, Gri ve Mavi vb. araç renkleri takip ediyor.

Genel olarak, nötr tonlar olan siyah, beyaz, gri ve gümüş gibi renkler en çok satılanlar arasında.

### Interior 

- En çok satan iç araç rengi Siyah. Satılan siyah araç sayısı 200.000'in üzerinde. 
- İkinci sırada en çok satan iç araç rengini Gri takip ediyor. Satılan araç sayısı 150.000'in üzerinde.
- Diğer en çok satan iç araç renkleri Bej, Krem ve - olarak isimlendirilmiş araç rengi vb. araç renkleri takip ediyor.

Genel olarak, açık tondaki renkler araç içinde en çok satılanlar arasında.

In [ ]:
def numSummary(dataframe, numericalCol, plot=False):
    quantiles = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95, 0.99, 1]
    summary = dataframe[numericalCol].describe(quantiles).T
    
    # Eksik veri sayısı ve oranı
    missing_count = dataframe[numericalCol].isnull().sum()
    missing_ratio = 100 * missing_count / len(dataframe)
    
    print(summary)
    print(f"Missing Count: {missing_count}")
    print(f"Missing Ratio: {missing_ratio:.2f}%")
    print("##########################################")
    
    if plot:
        plt.figure(figsize=(20, 8))
        dataframe[numericalCol].hist(color='green')
        plt.xlabel(numericalCol)
        plt.ylabel("Frequency")
        plt.title(numericalCol)
        plt.show(block=True)

for col in numCols:
    numSummary(df, col, True)

## Grafikleri Yorumlayalım: 

### Year

- Satılan araçların genel olarak 2000 ile 2015 yılları arasındaki model tarihlerine dağıldığı gözlemlenmektedir.

### Condition

- Satılan araçların durum puanlarının  arasındaki  tarihlerine dağıldığı gözlemlenmektedir.

### Odometer

- Satılan araçların yaptığı kilometre değerleri 0 ile 300.000 km arasında dağıldığı gözlemlenmektedir.

### MMR

- Satılan araçların pazar değerlerinin genel olarak 400 ile 44.000 dolar arasında dağıldığı gözlemlenmektedir.

### Selling price 

- Satılan araçların fiyatlarının genel olarak 400 ile 47.000 dolar arasında dağıldığı gözlemlenmektedir.

### Hedef olarak belirlediğimiz bir kategorik değişkenimizi numerik verilerimiz ile inceleyelim.

In [ ]:
def targetSummaryWithNum(dataframe, target, numCol):
    # Hedef değişkenin ortalamasını hesapla ve fiyatı en yüksek olandan sırala
    summary_df = dataframe.groupby(target).agg({numCol: "mean"}).sort_values(by=numCol, ascending=False)
    top_10 = summary_df.head(10)
    bottom_10 = summary_df.tail(10)
    print("Top 10:\n", top_10, end="\n\n\n")
    print("Bottom 10:\n", bottom_10, end="\n\n\n")

for col in numCols:
    targetSummaryWithNum(df, "make", col)



### Yıl (Year)
- En yeni araçlar: Tesla (2013.217), FIAT (2013.163), Ram (2012.785) gibi markalar en yeni araçlara sahip.
- En eski araçlar: Oldsmobile (2000.596), Ford Truck (2000.000), Chev Truck (2000.000) gibi markalar en eski araçlara sahip.
### Durum (Condition)
- En iyi durumda olan araçlar: Lamborghini (43.750), Audi (37.800), Ferrari (37.579) gibi markalar en iyi durumda olan araçlara sahip.
- En kötü durumda olan araçlar: Mercury (17.846), Plymouth (17.000), Oldsmobile (16.282) gibi markalar en kötü durumda olan araçlara sahip.
### Kilometre (Odometer)
- En yüksek kilometreye sahip araçlar: Dodge Truck (246997.000), Ford Truck (194084.667), Dot (191734.000) gibi markalar en yüksek kilometreye sahip.
- En düşük kilometreye sahip araçlar: Maserati (29630.737), Smart (28963.500), Aston Martin (26603.640) gibi markalar en düşük kilometreye sahip.
### MMR (Manheim Market Report)
- En yüksek MMR değerine sahip araçlar: Rolls-Royce (154294.118), Ferrari (129210.526), Lamborghini (111500.000) gibi markalar en yüksek MMR değerine sahip.
- En düşük MMR değerine sahip araçlar: Hyundai Truck (1875.000), Isuzu (1750.858), Oldsmobile (1095.055) gibi markalar en düşük MMR değerine sahip.
### Satış Fiyatı (Selling Price)
- En yüksek satış fiyatına sahip araçlar: Rolls-Royce (153488.235), Ferrari (127210.526), Lamborghini (112625.000) gibi markalar en yüksek satış fiyatına sahip.
- En düşük satış fiyatına sahip araçlar: Chev Truck (2000.000), Isuzu (1709.196), Oldsmobile (954.190) gibi markalar en düşük satış fiyatına sahip.

### Hedef olarak belirlediğimiz bir numerik değişkenimizi kategorik verilerimiz ile inceleyelim.

In [ ]:
def targetSummaryWithCat(dataframe, target, numCol):
    # Hedef değişkenin ortalamasını hesapla ve fiyatı en yüksek olandan sırala
    summary_df = pd.DataFrame({"TARGET_MEAN": dataframe.groupby(numCol)[target].mean()})
    sorted_summary_df = summary_df.sort_values(by="TARGET_MEAN", ascending=False).head(10)
    print(sorted_summary_df, end="\n\n\n")

for col in catCols:
    targetSummaryWithCat(df, "sellingprice", col)


### Marka (Make)
- En yüksek ortalama satış fiyatına sahip markalar: Rolls-Royce (153,488.235), Ferrari (127,210.526), Lamborghini (112,625.000) gibi lüks markalar en yüksek ortalama satış fiyatlarına sahip.
- En düşük ortalama satış fiyatına sahip markalar: Bu verilerde en düşük fiyatlı markalar belirtilmemiş, ancak önceki verilere göre Oldsmobile, Geo gibi markalar daha düşük fiyatlı olabilir.
### Model
- En yüksek ortalama satış fiyatına sahip modeller: 458 Italia (183,000.000), SLS AMG GT (156,500.000), i8 (154,222.222) gibi modeller en yüksek ortalama satış fiyatlarına sahip.
- En düşük ortalama satış fiyatına sahip modeller: Bu verilerde en düşük fiyatlı modeller belirtilmemiş, ancak önceki verilere göre daha düşük fiyatlı modeller olabilir.
### Gövde Tipi (Body)
- En yüksek ortalama satış fiyatına sahip gövde tipleri: Granturismo Convertible (79,041.667), GranTurismo Convertible (74,000.000), CTS-V Wagon (50,500.000) gibi gövde tipleri en yüksek ortalama satış fiyatlarına sahip.
- En düşük ortalama satış fiyatına sahip gövde tipleri: Bu verilerde en düşük fiyatlı gövde tipleri belirtilmemiş, ancak önceki verilere göre daha düşük fiyatlı gövde tipleri olabilir.
### Şanzıman (Transmission)
- En yüksek ortalama satış fiyatına sahip şanzıman tipleri: Sedan (13,686.364), sedan (13,650.000), automatic (13,540.407) gibi şanzıman tipleri en yüksek ortalama satış fiyatlarına sahip.
- En düşük ortalama satış fiyatına sahip şanzıman tipleri: Manual (11,211.448) en düşük ortalama satış fiyatına sahip.
### Eyalet (State)
- En yüksek ortalama satış fiyatına sahip eyaletler: ON (17,812.623), TN (17,009.744), PA (15,976.303) gibi eyaletler en yüksek ortalama satış fiyatlarına sahip.
- En düşük ortalama satış fiyatına sahip eyaletler: OH (14,406.619), MO (14,466.192) gibi eyaletler en düşük ortalama satış fiyatlarına sahip.
### Renk (Color)
- En yüksek ortalama satış fiyatına sahip renkler: Charcoal (16,247.497), black (15,509.005), brown (15,352.808) gibi renkler en yüksek ortalama satış fiyatlarına sahip.
- En düşük ortalama satış fiyatına sahip renkler: 2172 (14,050.000), 721 (14,100.000) gibi renkler en düşük ortalama satış fiyatlarına sahip.
### İç Mekan (Interior)
- En yüksek ortalama satış fiyatına sahip iç mekan renkleri: Off-white (29,045.990), red (23,789.087), brown (20,241.965) gibi iç mekan renkleri en yüksek ortalama satış fiyatlarına sahip.
- En düşük ortalama satış fiyatına sahip iç mekan renkleri: Purple (13,198.086), beige (13,348.511) gibi iç mekan renkleri en düşük ortalama satış fiyatlarına sahip.

## Korelasyon Matrisi

In [ ]:
figure, axes = plt.subplots(figsize=[18, 13])
sns.heatmap(df[numCols].corr(), annot=True, fmt=".2f", ax=axes, cmap="magma")
axes.set_title("Correlation Matrix", fontsize=20)
plt.show()

## Korelasyonlar
- Yıl ve Kilometre: Güçlü bir negatif korelasyon (-0.77) vardır, bu da daha yeni arabaların (daha yüksek 'yıl') daha düşük kilometreye (kilometre sayacı) sahip olduğunu gösterir.

- Yıl ve Satış Fiyatı: Orta düzeyde pozitif bir korelasyon (0.59) vardır, bu da daha yeni arabaların daha yüksek satış fiyatına sahip olduğunu gösterir.

- Kilometre ve Satış Fiyatı: Orta düzeyde negatif bir korelasyon (-0.58) vardır, bu da daha yüksek kilometreye sahip arabaların daha düşük satış fiyatına sahip olduğunu gösterir.

- MMR (Manheim Market Report) ve Satış Fiyatı: Çok güçlü bir pozitif korelasyon (0.98) vardır, bu da MMR değerinin satış fiyatının güçlü bir tahmincisi olduğunu gösterir.

- Durum ve Satış Fiyatı: Zayıf bir pozitif korelasyon (0.32) vardır, bu da daha iyi durumda olan arabaların daha yüksek satış fiyatına sahip olduğunu gösterir, ancak ilişki çok güçlü değildir.

- MMR ve Kilometre: Orta düzeyde negatif bir korelasyon (-0.59) vardır, bu da daha yüksek kilometreye sahip arabaların daha düşük MMR değerine sahip olduğunu gösterir.

- Yıl ve MMR: Orta düzeyde pozitif bir korelasyon (0.60) vardır, bu da daha yeni arabaların daha yüksek MMR değerine sahip olduğunu gösterir.

In [ ]:
# Seaborn pairplot fonksiyonunu kullanarak çift değişkenli ilişkileri bir ızgara (grid) şeklinde  görselleştirelim.
# sns.pairplot(df)


### Dağılımlar

- Yıl: Arabaların çoğu son yıllardan olup, yaklaşık 2000 yılından itibaren keskin bir artış görülmektedir.
- Durum: Durum değişkeni, veri noktalarının çoğu belirli seviyelerde kümelenmiş bazı ayrık değerlere sahip gibi görünmektedir, bu da durumun muhtemelen sıralı bir değişken (örneğin, 1, 2, 3 vb. - durum derecelendirmeleri) olduğunu göstermektedir.
- Kilometre: Dağılım sağa çarpık olup, çoğu aracın kilometresi daha düşüktür. Ancak, bazı araçlar çok yüksek kilometreye sahiptir.
- MMR: MMR değerlerinin dağılımı da sağa çarpık olup, daha düşük MMR değerlerine sahip arabaların sıklığı daha yüksektir.
- Satış Fiyatı: MMR'ye benzer şekilde, satış fiyatı dağılımı sağa çarpık olup, çoğu arabanın satış fiyatı daha düşüktür.

### İlişkiler 

- Yıl ve Kilometre: Burada açık bir negatif ilişki vardır. Daha yeni arabalar (daha yüksek 'yıl') daha düşük kilometreye sahip olma eğilimindedir. Bu, korelasyon matrisinde görülen güçlü negatif - korelasyonla uyumludur.
- Yıl ve Satış Fiyatı: Daha yeni arabaların daha yüksek satış fiyatlarına sahip olma eğilimi gösteren pozitif bir ilişki vardır. Bu, daha önce gözlemlenen pozitif korelasyonla tutarlıdır.
- Yıl ve MMR: Satış fiyatına benzer şekilde, daha yeni arabaların daha yüksek MMR değerlerine sahip olma eğilimi vardır.
- Durum ve Satış Fiyatı: Pozitif bir eğilim olsa da, dağılım, aynı durum derecesine sahip arabalar için satış fiyatlarında büyük bir değişkenlik olduğunu göstermektedir.
- Kilometre ve Satış Fiyatı: Daha yüksek kilometreye sahip arabaların daha düşük satış fiyatlarına sahip olma eğilimi gösteren açık bir negatif ilişki vardır. Ancak, bu ilişki doğrusal değil gibi görünüyor.
- MMR ve Satış Fiyatı: Çok güçlü bir pozitif doğrusal ilişki vardır, bu da MMR'nin satış fiyatının güçlü bir tahmincisi olduğunu doğrulamaktadır.
- Kilometre ve MMR: Satış fiyatıyla olan ilişkiye benzer şekilde, daha yüksek kilometreye sahip arabaların daha düşük MMR değerlerine sahip olduğu negatif bir ilişki vardır.

### Özet

- Satış Fiyatı için Güçlü Tahminciler: MMR, satış fiyatıyla en güçlü ve en doğrusal ilişkiye sahip olup, çok iyi bir tahminci haline gelmektedir. Yıl ve kilometre de önemli ilişkiler göstermektedir, ancak daha az güçlü ve doğrusal değildir.
- Dağılımlar: Hem satış fiyatı hem de MMR sağa çarpık olup, veri setindeki düşük fiyatlı araçların yüksek fiyatlı araçlardan daha fazla olduğunu göstermektedir.
- Değişken İlişkileri: Dağılım çizimleri, daha önce görülen korelasyonları doğrulayarak hem doğrusal hem de doğrusal olmayan ilişkileri vurgulamaktadır.

In [ ]:
# Boxplot kullanarak aykırı değerleri görselleştirelim.
for i in df.select_dtypes(include = "number").columns:
    plt.figure(figsize = (20,10))
    sns.boxplot(data = df, x = i, palette="viridis")
    plt.show()

## Grafikleri yorumlayalım:
### Condition değişkeni hariç diğer değişkenlerde aykırı değerlerin fazla olduğunu görüyoruz.

In [ ]:
fig, ax = plt.subplots(figsize=[20, 5])
sns.barplot(x='make', y='sellingprice', hue='transmission', data=df, ax=ax)
plt.xticks(rotation=90)
plt.show()

### Genel olarak resime bakınca otomatik vites araçların daha yüksek satış fiyatı olduğu görülüyor ancak bazı araçların manuel fiyatı yüksek burada yanlış bir veri olabilir veya aykırı olarak ele alıbiliriz.

In [ ]:
fig, ax = plt.subplots(figsize=[20, 5])
sns.barplot(x='color', y='sellingprice', data=df, ax=ax, palette='viridis', estimator=np.mean)
plt.xticks(rotation=90)
plt.show()

### Araçların satışında dış renklere göre fiyat farklılığı fazla olmadığı görülüyor. Renklere göre fiyatlar düzgün dağılmış diyebiliriz. 

In [ ]:
fig, ax = plt.subplots(figsize=[20, 5])
sns.barplot(x='interior', y='sellingprice',  data=df, ax=ax, palette='viridis', estimator=np.mean)
plt.xticks(rotation=90)
plt.show()

### Araçların satışında iç renklere göre fiyat farklılığında dalgalanmalar görülüyor. Renklere göre fiyatlar normal dağılmış diyebiliriz. 

In [ ]:
fig, ax = plt.subplots(figsize=[20, 5])
sns.barplot(x='condition', y='sellingprice', data=df, ax=ax, palette="viridis", estimator=np.mean)
plt.xticks(rotation=90)
plt.show()

### Grafiğe bakınca iki farklı sınıflandırma olduğunu görüyorum 1-5 arası ve 10-50 arası iki sınıftada üst değere yakın olanlar yüksek satış fiyatına sahip diyebiliriz.  

In [ ]:
fig, ax = plt.subplots(figsize=[20, 5])
sns.barplot(x='state', y='sellingprice', data=df, ax=ax, estimator=np.mean, palette='viridis')
plt.xticks(rotation=90)
plt.show()

### Genel olarak baktığımızda eyaletler arası pek bir fiyat farkı yok gibi. Düzgün dağılmış diyebiliriz. 

In [ ]:
fig, ax = plt.subplots(figsize=[20, 5])
sns.barplot(x='make', y='sellingprice', data=df, ax=ax, estimator=np.mean, palette='viridis')
plt.xticks(rotation=90)
plt.show()

### Fiyatların çoğu 20.000 doların altında. Coğunluğun az bir kısmı 20.000 doların üzerinde.

# Aykırı Gözlem Analizi 

In [ ]:
# Aykırı değerlerin baskılanması
def outlierThresholds(dataframe, variable, lowQuantile=0.10, upQuantile=0.90):
    quantileOne = dataframe[variable].quantile(lowQuantile)
    quantileThree = dataframe[variable].quantile(upQuantile)
    interquantile_range = quantileThree - quantileOne
    upLimit = quantileThree + 1.5 * interquantile_range
    lowLimit = quantileOne - 1.5 * interquantile_range
    return lowLimit, upLimit

# Aykırı değer kontrolü


def checkOutlier(dataframe, col_name):
    lowLimit, upLimit = outlierThresholds(dataframe, col_name)
    if dataframe[(dataframe[col_name] > upLimit) | (dataframe[col_name] < lowLimit)].any(axis=None):
        return True
    else:
        return False

# Aykırı değerlerin baskılanması


def replaceWithThresholds(dataframe, variable):
    lowLimit, upLimit = outlierThresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < lowLimit), variable] = lowLimit
    dataframe.loc[(dataframe[variable] > upLimit), variable] = upLimit


In [ ]:
for col in numCols:
    if col != "Sellingprice":
        print(col, checkOutlier(df, col))

### Condition değişkeni hariç aykırı değerlere rastladık. 

### Fonksiyon ile önceden belirlediğimiz alt ve üst limitler ile aykırı değerlerden kurtulalım.

In [ ]:
for col in numCols:
    if col != "Sellingprice":
        replaceWithThresholds(df, col)

In [ ]:
for col in numCols:
    if col != "Sellingprice":
        print(col, checkOutlier(df, col))

### Aykırı değerlerin baskıladık

# Yanlış ve az bilgi içeren sütunları silelim

In [ ]:
df = df[df['state'].apply(lambda x: len(x) <= 3)]
state_counts = df['state'].value_counts()
state_counts.plot(kind='bar')
plt.title('State Counts')
plt.xlabel('States')
plt.ylabel('Count')
plt.show()

### Tuhaf isimlendirilmiş eyaletleri sayıları çok az olduğu için sildik. Daha temiz ve okunur veri elde ettik.

In [ ]:
df = df[~df['transmission'].str.lower().isin(['sedan'])]
state_counts = df['transmission'].value_counts()
state_counts.plot(kind='bar')
plt.title('Transmission Counts')
plt.xlabel('Transmission')
plt.ylabel('Count')
plt.show()


### Sedan olarak isimlendirilmiş vites sınıfları sildik.

In [ ]:
df['body'] = df['body'].str.lower()
state_counts = df['body'].value_counts()
state_counts.plot(kind='bar')
plt.title('Body Counts')
plt.xlabel('Body')
plt.ylabel('Count')
plt.show()

### Body değişkeninde tüm sınıfları küçük harfe çevirdik böylece aynı sınıfa ait olan veriler tekrarlamamış oldu.

In [ ]:
df['trim'] = df['trim'].str.lower()
state_counts = df['trim'].value_counts()


In [ ]:
def is_numeric(value):
    try:
        float(value)  
        return True
    except ValueError:
        return False

df = df[~df['color'].apply(is_numeric)]
state_counts = df['color'].value_counts()
state_counts.plot(kind='bar')
plt.title('Color Counts')
plt.xlabel('Color')
plt.ylabel('Count')
plt.show()

### Renklerin sayısal karsılıkları belli olmadığı için sildik.

In [ ]:
df['color'] = df['color'].replace('—', 'unknown')
class_counts = df['color'].value_counts()
class_counts

### - ile isimlendirilmiş sınıfı unknown olarak güncelledik.

In [ ]:
df['interior'] = df['interior'].replace('—', 'unknown')
class_counts = df['interior'].value_counts()
class_counts

### Aynı işlemi iç renk için de yaptık.

# Eksik Değer Analizi

In [ ]:
def missingValuesTable(dataframe, naName=False):
    naColums = [
        col for col in dataframe.columns if dataframe[col].isnull().sum() > 0]
    nMiss = dataframe[naColums].isnull().sum().sort_values(ascending=False)
    ratio = (dataframe[naColums].isnull().sum() /
             dataframe.shape[0] * 100).sort_values(ascending=False)
    missingDf = pd.concat([nMiss, np.round(ratio, 2)],
                          axis=1, keys=['n_miss', 'ratio'])
    print(missingDf, end='\n')
    if naName:
        return naColums


missingValuesTable(df)


### Eksik değerlerin oranını elde ettik en çok tranmission verisinde eksik var. 

In [ ]:
df.loc[:,'make'] = df.loc[:,['make']].fillna(df['make'].mode()[0])
df.loc[:,'model'] = df.loc[:,['model']].fillna(df['model'].mode()[0])
df.loc[:,'body'] = df.loc[:,['body']].fillna(df['body'].mode()[0])
df.loc[:,'trim'] = df.loc[:,['trim']].fillna(df['trim'].mode()[0])
df.loc[:,'color'] = df.loc[:,['color']].fillna(df['color'].mode()[0])
df.loc[:,'interior'] = df.loc[:,['interior']].fillna(df['interior'].mode()[0])
df.loc[:,'transmission'] = df.loc[:,['transmission']].fillna(df['transmission'].mode()[0])

df.loc[:,'condition'] = df.loc[:,['condition']].fillna(df['condition'].mean())
df.loc[:,'odometer'] = df.loc[:,['odometer']].fillna(df['odometer'].mean())
df.loc[:,'mmr'] = df.loc[:,['mmr']].fillna(df['mmr'].mean())

df.dropna(subset=['sellingprice'],axis='rows',inplace=True)

In [ ]:
missingValuesTable(df)

### Eksik verileri çeşitli yöntemlerle tamamladık. 

# Encoding 

In [ ]:
lbl_enc = LabelEncoder()
for col in df.columns:
    if df[col].dtype == "object":
        df[col] = lbl_enc.fit_transform(df[col])
        


In [ ]:
missingValuesTable(df)
df.drop(['seller','vin','saledate', 'model', 'trim','body'],axis=1,inplace=True)

In [703]:
y = df["sellingprice"]
x = df.drop(["sellingprice"],axis=1)

In [705]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import silhouette_score, davies_bouldin_score
import numpy as np

models = {
    'Linear Regression': LinearRegression(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor()
}


results = {}
for name, model in models.items():
    scores = cross_val_score(model, x, y, cv=5, scoring='neg_mean_squared_error')
    results[name] = {
        'Mean Score': np.mean(scores),
        'Standard Deviation': np.std(scores)
    }

for name, metrics in results.items():
    print(f"{name}:")
    print(f"  Mean Score: {metrics['Mean Score']:.3f}")
    print(f"  Standard Deviation: {metrics['Standard Deviation']:.3f}")
    print()


In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

kmeans = KMeans(n_clusters=number_of_clusters)
cluster_labels = kmeans.fit_predict(x)

true_labels = y

f1 = f1_score(true_labels_encoded, cluster_labels_encoded, average='weighted')
print(f'F1 Score: {f1:.3f}')

silhouette_avg = silhouette_score(x, cluster_labels)
davies_bouldin_avg = davies_bouldin_score(x, cluster_labels)

print(f'Silhouette Score: {silhouette_avg:.3f}')
print(f'Davies-Bouldin Score: {davies_bouldin_avg:.3f}')
